In [87]:
from selenium import webdriver
import chromedriver_binary
from bs4 import BeautifulSoup
import pandas as pd
import sys


In [88]:
opt = webdriver.ChromeOptions()
opt.add_argument('--headless')
opt.add_argument('--ignore-certificate-errors')
opt.add_argument('--allow-running-insecure-content')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
opt.add_argument(f'user-agent={user_agent}')

In [89]:

def chewy(k,out_queue):
    
    print(1)
    # chewy
    driver_chewy = webdriver.Chrome(options=opt)
    # chewy
    
    url_chewy = "https://www.chewy.com/s?query=cat%20can%20"+k+"&nav-submit-button="
    driver_chewy.get(url_chewy)
    website_chewy = []
    prices_chewy = []
    names_chewy = []

    content_chewy = driver_chewy.page_source
    soup_chewy = BeautifulSoup(content_chewy,"html.parser")
    n = soup_chewy.findAll('div',attrs={'class':'ProductListing_kibProductCard__3KgKt js-tracked-product kib-product-card'})
    if len(n) != 0:
        for a in n[:min(5,len(n))-1]:
            price=a['data-price']
            name=a['data-name']
        
            website_chewy.append('chewy')
            prices_chewy.append(float(price))
            names_chewy.append(name.strip()) 

    else:
        website_chewy = 'chewy'
        prices_chewy = 'no result'
        names_chewy = 'no result'
    
    
    d = {'Name':names_chewy,'Price':prices_chewy,'Sold by':website_chewy}
    df = pd.DataFrame(d)

    out_queue.put([df,url_chewy])
    
    

    

In [97]:

def petsmart(k,out_queue):    #petsmart
    
    print(2)
    driver_petsmart = webdriver.Chrome(options=opt)
    url_petsmart = "https://www.petsmart.com/search/?q=cat%20wet%20food%20"+k+"&ps=undefined"
    driver_petsmart.get(url_petsmart)
    
    website_petsmart = []
    prices_petsmart = []
    names_petsmart = []

    content_petsmart = driver_petsmart.page_source
    soup_petsmart = BeautifulSoup(content_petsmart,"html.parser")
    n = soup_petsmart.findAll('li',attrs={'class':'grid-tile gtm-grid-tile col-md-4 col-sm-12'})
    if len(n) != 0:
        for a in n[:min(5,len(n))-1]:
            price=a.find('span', attrs={'class':'price-regular'})
            if not price: 
                price=a.find('span', attrs={'class':'price-sales'})
            name=a.find('div', attrs={'class':'product-name'})
        
            website_petsmart.append('petsmart')
            prices_petsmart.append(float(price["data-gtm-price"]))
            names_petsmart.append(name.text.strip()) 

    else:
        website_petsmart = 'petsmart'
        prices_petsmart = 'no result'
        names_petsmart = 'no result'

    
    d = {'Name':names_petsmart,'Price':prices_petsmart,'Sold by':website_petsmart}
    df = pd.DataFrame(d)

    out_queue.put([df,url_petsmart])
    

In [98]:

def petco(k,out_queue):    #petco
    
    print(3)
    driver_petco = webdriver.Chrome(options=opt)



    url_petco = "https://www.petco.com/shop/SearchDisplay?categoryId=&storeId=10151&catalogId=10051&langId=-1&sType=SimpleSearch&resultCatEntryType=2&showResultsPage=true&searchSource=Q&pageView=&beginIndex=0&pageSize=48&fromPageValue=search&searchKeyword=&searchTerm="+k
    driver_petco.get(url_petco)

    website_petco = []
    prices_petco = []
    names_petco = []

    content_petco = driver_petco.page_source
    soup_petco = BeautifulSoup(content_petco,"html.parser")
    n = soup_petco.findAll('div',attrs={'class':'product-info'})
    if len(n) != 0:
        for a in n[:min(5,len(n))-1]:
            price=a.find('span', attrs={'class':'product-price-promo'})
            name=a.find('div', attrs={'class':'product-name'})
            website_petco.append('Petco')
            prices_petco.append(price.text.strip())
            names_petco.append(name.text.strip()) 
        
        names_petco = [a.replace("\n", " ") for a in names_petco]
    if len(n) == 0:
        website_petco = ['petco']
        prices_petco = ['no result']
        names_petco = ['no result']

    


    
    
    d = {'Name':names_petco,'Price':prices_petco,'Sold by':website_petco}
    df = pd.DataFrame(d)

    out_queue.put([df,url_petco])
    

In [109]:
from threading import Thread
import queue


def search_price(k):
    k = k.replace(' ','%20')
    my_queue1 = queue.Queue()
    my_queue2 = queue.Queue()
    my_queue3 = queue.Queue()
    
    ret_1 = Thread(target = petsmart,args=(k,my_queue1))
    ret_2 = Thread(target = chewy,args=(k,my_queue2))
    ret_3 = Thread(target = petco,args=(k,my_queue3))
    ret_1.start()
    ret_2.start()
    ret_3.start()
    
    ret1 = my_queue1.get()
    ret2 = my_queue2.get()
    ret3 = my_queue3.get()
    
    ret_1.join()
    ret_2.join()
    ret_3.join()
    
    result = [pd.concat([ret1[0],ret2[0],ret3[0]]),ret1[1],ret2[1],ret3[1]]
    return result
        


In [110]:
search_price('blue')


2
1
3


[                                                Name            Price Sold by
 0  Blue Buffalo Wilderness Wild Delights Chicken ...             30.0   chewy
 1  Blue Buffalo Wilderness Duck Grain-Free Canned...             30.0   chewy
 2  Blue Buffalo Wilderness Wild Delights Minced C...            42.89   chewy
 3  Blue Buffalo Tastefuls Tuna Entrée in Gravy Fl...            26.26   chewy
 0  Blue Buffalo Wilderness Natural Adult High Pro...  $19.98 - $79.99   Petco
 1  Blue Buffalo Blue Life Protection Formula Adul...  $16.98 - $62.98   Petco
 2  Blue Buffalo Wilderness Natural Adult High Pro...  $13.99 - $39.98   Petco
 3  Blue Buffalo Wilderness Natural Adult High Pro...  $20.98 - $81.99   Petco,
 'https://www.petsmart.com/search/?q=cat%20wet%20food%20blue&ps=undefined',
 'https://www.chewy.com/s?query=cat%20can%20blue&nav-submit-button=',
 'https://www.petco.com/shop/SearchDisplay?categoryId=&storeId=10151&catalogId=10051&langId=-1&sType=SimpleSearch&resultCatEntryType=2&showRes